In [1]:
import time
import pandas as pd
import numpy as np
import sys
sys.path.append("..")
from algorithm import a_star_algorithm
from arm_state import ArmState, set_arm_length
from hold import Hold
from human_state import HumanState, set_leg_spread
from leg_state import LegState, set_leg_length
from utils import Side
from search_node import Search_Node

### Walls

In [2]:
# Define wall holds and goal state positions.
wall_holds: list[Hold] = [
    Hold(x,y) for x in range(1, 11, 2) for y in range(1, 21, 2)
    ]

goal_state_positions = [np.array([9,19]), np.array([9,19])]

In [3]:
# Define wall holds and goal state positions to small wall.
small_wall_holds: list[Hold] = [
    Hold(x,y) for x in range(1, 8, 2) for y in range(1, 14, 2)
    ]

goal_state_positions = [np.array([7,13]), np.array([7,13])]

In [4]:
# Define wall holds and goal state positions to big wall.
big_wall_holds: list[Hold] = [
    Hold(x,y) for x in range(1, 21, 3) for y in range(1, 26, 3)
    ]
big_wall_holds.append(Hold(3, 1))
big_wall_holds.append(Hold(5, 1))
big_wall_holds.append(Hold(5, 7))

goal_state_positions = [np.array([19,25]), np.array([19,25])]

### Climber

In [5]:
# Define starting left leg configuration.
starting_left_foot_hold = Hold(3, 1)
starting_left_knee_position = np.array([2,2.75])
starting_left_hip_position = np.array([3, 4])

# Define starting right leg configuration.
starting_right_foot_hold = Hold(5, 1)
starting_right_knee_position = np.array([6, 2.75])
starting_right_hip_position = np.array([5, 4])

# Define starting left arm configuration.
starting_left_hand_hold = Hold(1, 7)
starting_left_shoulder_position = np.array([3, 5.5])

# Define starting right arm configuration.
starting_right_hand_hold = Hold(5,7)
starting_right_shoulder_position = np.array([5, 5.5])

# Define initial human state based on starting body configurations.
starting_left_leg_state = LegState(Side.LEFT, starting_left_foot_hold, starting_left_knee_position,
                                    starting_left_hip_position)
starting_right_leg_state = LegState(Side.RIGHT, starting_right_foot_hold, starting_right_knee_position,
                                    starting_right_hip_position)
starting_left_arm_state = ArmState(Side.LEFT, starting_left_hand_hold, starting_left_shoulder_position)
starting_right_arm_state = ArmState(Side.RIGHT, starting_right_hand_hold, starting_right_shoulder_position)
starting_human_state = HumanState(starting_left_leg_state, starting_right_leg_state, starting_left_arm_state,
                                    starting_right_arm_state)

# Define body physique constants.
set_leg_spread(np.float_(np.float64(2.0943951)))
set_arm_length(np.float_(np.float64(2.5)))
set_leg_length(starting_right_foot_hold, starting_right_knee_position, starting_right_hip_position)

# heuristic function

In [7]:
def heuristic(wall_holds, calculate_heuristic=None):
    generated_nodes=[]
    # start timer to mesure time
    import time
    start = time.time()
    # Run A* algorithm.
    if calculate_heuristic is not None:
        path:list[Search_Node] | None = a_star_algorithm(starting_human_state, goal_state_positions, wall_holds, calculate_heuristic, generated_nodes)
    else:
        path:list[Search_Node] | None = a_star_algorithm(starting_human_state, goal_state_positions, wall_holds, generated_nodes=generated_nodes)
    # stop timer
    end = time.time()
    num_generated_nodes = len(generated_nodes)
    time =  (end - start) / 60
    return num_generated_nodes, time

# Heuristic of 0

In [19]:
def calculate_heuristic(node: Search_Node, goal_state_positions):
    return 0

In [20]:
num_generated_node_small_0, time_small_0 = heuristic(small_wall_holds,calculate_heuristic)
num_generated_node_small_0, time_small_0

(37346, 8.191353273391723)

In [21]:
num_generated_node_0, time_0 = heuristic(wall_holds, calculate_heuristic)
num_generated_node_0, time_0

(96421, 33.65082120895386)

In [22]:
num_generated_node_big_0, time_big_0 = heuristic(big_wall_holds, calculate_heuristic)
num_generated_node_big_0, time_big_0

(7677, 2.1510249813397726)

# our Heurstic

In [7]:
num_generated_node_small_our, time_small_our = heuristic(small_wall_holds)
num_generated_node_small_our, time_small_our

(37351, 9.00612288316091)

In [16]:
num_generated_node_medium_our, time_medium_our = heuristic(wall_holds)
num_generated_node_medium_our, time_medium_our

(95899, 29.936403981844585)

In [15]:
num_generated_node_big_our, time_big_our = heuristic(big_wall_holds)
num_generated_node_big_our, time_big_our

(7240, 2.696656012535095)

In [17]:
num_generated_node_medium_our, time_medium_our = 95899, 29.936403981844585

num_generated_node_big_our, time_big_our = 7240, 2.696656012535095


# Weight A*

0.9 * h + 0.1 * g

In [8]:
num_generated_node_small_weight, time_small_weight = heuristic(small_wall_holds)
num_generated_node_small_weight, time_small_weight

(37364, 8.866348612308503)

In [9]:
num_generated_node_medium_weight, time_medium_weight = heuristic(wall_holds)
num_generated_node_medium_weight, time_medium_weight

(95468, 28.95693159500758)

In [10]:
num_generated_node_big_weight, time_big_weight = heuristic(big_wall_holds)
num_generated_node_big_weight, time_big_weight

(7240, 2.239032538731893)

# Compare

In [19]:
# Create a dictionary with the data
data = {
    'wall size': ['small', 'small', 'small', 'medium', 'medium', 'medium', 'large', 'large', 'large'],
    'heuristic': ['our heuristic', 'weight heuristic', 'no heuristic'] * 3,
    'generate node': [num_generated_node_small_our, num_generated_node_small_weight, num_generated_node_small_0,
                    num_generated_node_medium_our, num_generated_node_medium_weight, num_generated_node_0,
                    num_generated_node_big_our, num_generated_node_big_weight, num_generated_node_big_0],
    'time(minute)': [time_small_our, time_small_weight, time_small_0,
                    time_medium_our, time_medium_weight, time_0,
                    time_big_our, time_big_weight, time_big_0]  # Assuming time is in seconds
}

df = pd.DataFrame(data)

In [20]:
df

,wall size,heuristic,generate node,time(minute)
0,small,our heuristic,37351,9.006123
1,small,weight heuristic,37364,8.866349
2,small,no heuristic,37346,8.191353
3,medium,our heuristic,95899,29.936404
4,medium,weight heuristic,95468,28.956932
5,medium,no heuristic,96421,33.650821
6,large,our heuristic,7240,2.696656
7,large,weight heuristic,7240,2.239033
8,large,no heuristic,7677,2.151025
